# <font color="red">Warning: I don't have time over Christmas to comment this properly, or describe what I'm doing, so the following notebook shouldn't be taken as a good way of doing things!</font>

It's taken an unusually long time to get a proper search question. I'm guessing the first part will be reasonably tractable.

Trying to write a search routine with mutable data structures is an absolute pain in the neck, so I'm going to try out this `pyrsistent` library that's been mentioned to me:

In [1]:
import pyrsistent as pyr

In [2]:
testInput='''
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
'''

Might be better as a set, but there's always the chance that some parts are repeated:

In [3]:
l=[(int(x), int(y)) 
   for [x, y] in [nl.split('/') 
                  for nl in testInput.strip().split('\n')]]

parts_pv=pyr.pvector(l)


In [4]:
[x for x in parts_pv if x[0]==2 or x[1]==2]

[(0, 2), (2, 2), (2, 3)]

In [5]:
pyr.v((1, 0), (2, 2))

pvector([(1, 0), (2, 2)])

In [6]:
for i in l:
    print(i)

(0, 2)
(2, 2)
(2, 3)
(3, 4)
(3, 5)
(0, 1)
(10, 1)
(9, 10)


In [7]:
firstState_pm=pyr.m(seeking=0,
                    parts=parts_pv,
                    bridge=pyr.v())
firstState_pm

pmap({'seeking': 0, 'parts': pvector([(0, 2), (2, 2), (2, 3), (3, 4), (3, 5), (0, 1), (10, 1), (9, 10)]), 'bridge': pvector([])})

In [8]:
def next_states(agendaIn_pv):
    '''
    Takes an agenda of states.
    Returns a pair of the current state
    and an updated agenda.
    '''
    # Take the first item off the agenda
    nextItem_pm=agendaIn_pv[0]
    
    agendaOut_pv=agendaIn_pv.delete(0)
    
    # Find the list of parts which can attach to the 
    # current exposed part:
    validNextStates_ls=[x for x in nextItem_pm['parts']
                        if x[0]==nextItem_pm['seeking'] or x[1]==nextItem_pm['seeking']]
    
    # For each of these states, create a new agenda item:
    newAgendaItems_pv=pyr.v()
    
    for (x, y) in validNextStates_ls:
        if x==nextItem_pm['seeking']:
            newAgendaItems_pv=newAgendaItems_pv.append(pyr.m(seeking=y,
                                           parts=nextItem_pm['parts'].remove((x, y)),
                                           bridge=nextItem_pm['bridge'].append((x, y))))
        else:
            newAgendaItems_pv=newAgendaItems_pv.append(pyr.m(seeking=x,
                                           parts=nextItem_pm['parts'].remove((x, y)),
                                           bridge=nextItem_pm['bridge'].append((x, y))))

    return (nextItem_pm, agendaOut_pv.extend(newAgendaItems_pv))

In [9]:
next_states(pyr.v(firstState_pm))

(pmap({'seeking': 0, 'parts': pvector([(0, 2), (2, 2), (2, 3), (3, 4), (3, 5), (0, 1), (10, 1), (9, 10)]), 'bridge': pvector([])}),
 pvector([pmap({'seeking': 2, 'parts': pvector([(2, 2), (2, 3), (3, 4), (3, 5), (0, 1), (10, 1), (9, 10)]), 'bridge': pvector([(0, 2)])}), pmap({'seeking': 1, 'parts': pvector([(0, 2), (2, 2), (2, 3), (3, 4), (3, 5), (10, 1), (9, 10)]), 'bridge': pvector([(0, 1)])})]))

Might make things a shade more readable if we define a function `bridge_strength` to apply to states:

In [10]:
def bridge_strength(stateIn):
    '''
    Return the strength of the bridge in the current state
    '''
    return sum([x+y for (x, y) in stateIn['bridge']])

OK, now we should just be able to do a DFS to return the strongest bridge.

In [11]:
parts_pv=pyr.pvector([(int(x), int(y)) 
                      for [x, y] in [nl.split('/')
                                     for nl in testInput.strip().split('\n')]])

firstState_pm=pyr.m(seeking=0,
                    parts=parts_pv,
                    bridge=pyr.v())

# Check the first state:
firstState_pm



pmap({'seeking': 0, 'parts': pvector([(0, 2), (2, 2), (2, 3), (3, 4), (3, 5), (0, 1), (10, 1), (9, 10)]), 'bridge': pvector([])})

Now just run to exhaustion:

In [12]:
agenda_pv=pyr.v(firstState_pm)

strongestBridge_i=0

while len(agenda_pv):
    (nextState_pm, agenda_pv)=next_states(agenda_pv)
    if bridge_strength(nextState_pm)>strongestBridge_i:
        strongestBridge_i=bridge_strength(nextState_pm)
    
assert strongestBridge_i==31

Good, now apply to the puzzle input:

In [14]:
with open('data/day24.txt') as fIn:
    parts_pv=pyr.pvector([(int(x), int(y)) 
                          for [x, y] in [nl.split('/')
                                         for nl in fIn.read().strip().split('\n')]])


firstState_pm=pyr.m(seeking=0,
                    parts=parts_pv,
                    bridge=pyr.v())

agenda_pv=pyr.v(firstState_pm)

strongestBridge_i=0

while len(agenda_pv):
    (nextState_pm, agenda_pv)=next_states(agenda_pv)
    if bridge_strength(nextState_pm)>strongestBridge_i:
        strongestBridge_i=bridge_strength(nextState_pm)
        print(strongestBridge_i)
    
strongestBridge_i



9
46
106
161
172
186
222
226
237
280
284
294
322
327
352
380
394
419
430
452
461
472
507
519
530
539
549
560
597
607
639
649
660
664
697
699
706
728
729
749
757
764
806
828
841
906
907
926
941
950
951
963
965
983
1004
1008
1041
1043
1050
1073
1074
1088
1101
1108
1117
1124
1136
1152
1158
1159
1166
1173
1182
1209
1224
1231
1251
1260
1309
1344
1353
1379
1394
1401
1411
1414
1430
1479
1514
1536
1538
1544
1559
1594
1626
1631
1653
1673
1688
1696
1703
1764
1770
1784
1790
1846
1852
1866
1868


1868

That took ages... probably worth trying to speed it up for the next part.